In [1]:
import os
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from parse import *
import seaborn as sns

In [2]:
CHUNKSIZE = 100000

TRAIN_PATH        = "../train_StationPathInfo.csv"
TRAIN_PATH_EX     = "../train_StationPathInfoEx.csv"
TEST_PATH         = "../test_StationPathInfo.csv"
TEST_PATH_EX      = "../test_StationPathInfoEx.csv"

TRAIN_DATE        = "../train_date.csv"
TRAIN_NUMERIC     = "../train_numeric.csv"
TRAIN_CATEGORICAL = "../train_categorical.csv"
TEST_DATE         = "../test_date.csv"
TEST_NUMERIC      = "../test_numeric.csv"
TEST_CATEGORICAL  = "../test_categorical.csv"

SEED = 0
CHUNKSIZE = 50000
NROWS = 1200000

ID_COLUMN = 'Id'
TARGET_COLUMN = 'Response'
    
    

In [3]:
chunk_train_num = pd.read_csv(TRAIN_NUMERIC, nrows=NROWS, chunksize=CHUNKSIZE)
chunk_test_num  = pd.read_csv(TEST_NUMERIC, nrows=NROWS, chunksize=CHUNKSIZE)
chunk_train_cat = pd.read_csv(TRAIN_CATEGORICAL, nrows=NROWS, chunksize=CHUNKSIZE)
chunk_test_cat  = pd.read_csv(TEST_CATEGORICAL, nrows=NROWS, chunksize=CHUNKSIZE)

df_train_num = chunk_train_num.get_chunk(10)
df_test_num  = chunk_test_num.get_chunk(10)
df_train_cat = chunk_train_cat.get_chunk(10)
df_test_cat  = chunk_test_cat.get_chunk(10)

df_train_num.head()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [4]:
STATION_NUM = 52

num_cols = df_train_num.columns
cat_cols = df_train_cat.columns

num_feature_list = []
cat_feature_list = []

for cur_station_id in range(STATION_NUM):
    num_feature_list_one = []
    cat_feature_list_one = []
    for column_name in num_cols:
        if column_name != 'Id':
            r = parse("L{}_S{}_F{}", column_name)
            if r is not None:
                col_line_id    = int(r[0])
                col_station_id = int(r[1])
                if cur_station_id == col_station_id:
                    num_feature_list_one.append(column_name)
    for column_name in cat_cols:
        if column_name != 'Id':
            r = parse("L{}_S{}_F{}", column_name)
            if r is not None:
                col_line_id    = int(r[0])
                col_station_id = int(r[1])
                if cur_station_id == col_station_id:
                    cat_feature_list_one.append(column_name)
        
    num_feature_list.append(num_feature_list_one)
    cat_feature_list.append(cat_feature_list_one)

In [10]:
for i in range(STATION_NUM):
    print(str(i) + '   ' + str(len(num_feature_list[i])) + '   ' + str(len(cat_feature_list[i])))

0   12   0
1   2   4
2   9   18
3   9   18
4   2   6
5   2   0
6   3   10
7   3   0
8   3   0
9   12   39
10   12   39
11   12   26
12   12   0
13   2   0
14   9   18
15   9   9
16   2   6
17   2   0
18   3   10
19   3   0
20   3   0
21   14   45
22   14   45
23   14   30
24   229   660
25   284   567
26   14   53
27   14   53
28   14   53
29   53   63
30   68   204
31   4   8
32   1   3
33   10   0
34   4   0
35   8   18
36   8   8
37   4   0
38   3   6
39   4   8
40   8   0
41   14   0
42   0   24
43   8   24
44   8   8
45   5   0
46   0   3
47   11   33
48   6   0
49   7   21
50   7   0
51   4   0


S24, S25, S30 の feature が多いので、ここを外すと普通に扱えそう…
だが、そんなことはなかった。
feature importance で上位に来る、29, 30, 31, 33 の numeric でやってみる。

In [29]:
use_station_list = [29, 30, 31, 33]

use_cat_columns = ['Id']
use_num_columns = ['Id']

for i in range(STATION_NUM):
    if i in use_station_list:
        use_cat_columns += cat_feature_list[i]
        use_num_columns += num_feature_list[i]

print(use_cat_columns)
print(use_num_columns)
print(len(use_cat_columns))
print(len(use_num_columns))

['Id', 'L3_S29_F3317', 'L3_S29_F3320', 'L3_S29_F3323', 'L3_S29_F3326', 'L3_S29_F3329', 'L3_S29_F3332', 'L3_S29_F3335', 'L3_S29_F3338', 'L3_S29_F3341', 'L3_S29_F3344', 'L3_S29_F3347', 'L3_S29_F3350', 'L3_S29_F3353', 'L3_S29_F3356', 'L3_S29_F3359', 'L3_S29_F3362', 'L3_S29_F3364', 'L3_S29_F3366', 'L3_S29_F3369', 'L3_S29_F3372', 'L3_S29_F3375', 'L3_S29_F3378', 'L3_S29_F3381', 'L3_S29_F3384', 'L3_S29_F3387', 'L3_S29_F3390', 'L3_S29_F3392', 'L3_S29_F3394', 'L3_S29_F3397', 'L3_S29_F3400', 'L3_S29_F3403', 'L3_S29_F3406', 'L3_S29_F3409', 'L3_S29_F3411', 'L3_S29_F3414', 'L3_S29_F3416', 'L3_S29_F3418', 'L3_S29_F3420', 'L3_S29_F3423', 'L3_S29_F3426', 'L3_S29_F3429', 'L3_S29_F3432', 'L3_S29_F3435', 'L3_S29_F3438', 'L3_S29_F3441', 'L3_S29_F3444', 'L3_S29_F3446', 'L3_S29_F3448', 'L3_S29_F3451', 'L3_S29_F3454', 'L3_S29_F3457', 'L3_S29_F3460', 'L3_S29_F3463', 'L3_S29_F3466', 'L3_S29_F3469', 'L3_S29_F3472', 'L3_S29_F3475', 'L3_S29_F3478', 'L3_S29_F3481', 'L3_S29_F3484', 'L3_S29_F3487', 'L3_S29_F3490', '

In [30]:
df_cat_train = pd.read_csv(TEST_CATEGORICAL,  
                            usecols=use_cat_columns, nrows=NROWS)
df_cat_test  = pd.read_csv(TEST_CATEGORICAL,  
                            usecols=use_cat_columns, nrows=NROWS)

C:\Users\Tomonobu\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1773,1774,1775,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815,1816,1817,1818,1819,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837,1838,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1853,1854,1855,1856,1857,1858,1859,1860,1861,1862,1863,1864,1865,1866,1867,1868,1869,1870,1871,1872,1873,1874,1875,1876,1877,1878,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913,1914,1915,1916,1917,1918,1919,1920,1921,1922,1923,1924,1925,1926,1927,1928,1929,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945,1946,1947,1948,1949,1950,19

In [31]:
df_num_train = pd.read_csv(TEST_NUMERIC,  
                            usecols=use_num_columns, nrows=NROWS)
df_num_test  = pd.read_csv(TEST_NUMERIC,  
                            usecols=use_num_columns, nrows=NROWS)

In [32]:
# 読み出す。

df_train_bin = pd.read_csv(TRAIN_PATH)
df_test_bin  = pd.read_csv(TEST_PATH)
df_pass = pd.concat([df_train_bin, df_test_bin])

In [33]:
# 時系列でソートしたヒートマップを出してみる。

TEST_START_TIME      = "../test_StartEndTime.csv"
TRAIN_START_TIME     = "../train_StartEndTime.csv"

df_start_train = pd.read_csv(TRAIN_START_TIME)
df_start_train = df_start_train.ix[:,['Id','StartTime','EndTime', 'Response']]
df_start_test = pd.read_csv(TEST_START_TIME)
df_start_test['Response'] = -1


C:\Users\Tomonobu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [34]:
df_start_test.head()
df_start = pd.concat([df_start_train, df_start_test])
df_start.head()

,Id,StartTime,EndTime,Response
0,4,82.24,87.29,0
1,6,1313.12,1315.75,0
2,7,1618.70,1624.42,0
3,9,1149.20,1154.16,0
4,11,602.64,606.02,0


In [35]:
df_all = pd.merge(df_start, df_pass, on='Id',  how='left')
df_all = df_all.fillna(-1500)

In [36]:
df_all = pd.merge(df_all, df_num_train, on='Id', how='left')
df_all = pd.merge(df_all, df_num_test,  on='Id', how='left')


In [38]:
df_all = df_all.fillna(-1500)
df_all.head

<bound method NDFrame.head of               Id  StartTime  EndTime  Response  L0_S0_D1  L0_S1_D26  \
0              4      82.24    87.29         0         1          1   
1              6    1313.12  1315.75         0         0          0   
2              7    1618.70  1624.42         0         1          1   
3              9    1149.20  1154.16         0         1          1   
4             11     602.64   606.02         0         1          1   
5             13    1331.66  1339.73         0         1          1   
6             14    1662.63  1664.04         0         0          0   
7             16     791.22   804.36         0         0          0   
8             18     517.64   518.08         0         1          1   
9             23     156.27   157.89         0         0          0   
10            26    1104.78  1105.95         0         1          1   
11            27     392.85   401.41         0         1          1   
12            28      55.44    62.10         0 

In [39]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, log_loss, make_scorer

def calc_mcc(cf_mat):
    tn, fp, fn, tp = cf_mat.ravel()
    print(tn, fp, fn, tp)
    mcc = (tp * tn - fp * fn) / np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    return mcc

def mcc_scorer(y_true, y_pred):
    cf_mat = confusion_matrix(y_true, y_pred)
    return calc_mcc(cf_mat)

def separate_X_y(df):
    X = df.drop(['Response'], axis=1)
    y = df['Response']
    return X, y

def train_with_r_forest(df):
    X_train, y_train = separate_X_y(df)

    cl_weight = {0:0.3, 1:0.7}
    
    # Grid Search
    params={'max_depth': [25],
            'subsample': [0.95],
            'colsample_bytree': [1.0],
            'scale_pos_weight': [2]
    }

    xgb_model = xgb.XGBClassifier()
    gs = GridSearchCV(xgb_model,
                      params,
                      cv=5,
                      scoring={'mcc' : make_scorer(mcc_scorer)},
                      n_jobs=1,
                      verbose=2,
                      refit='mcc')

    gs.fit(X_train, y_train)
#rf = RandomForestClassifier(max_depth=10,n_estimators=50, random_state=33)
#    rf.fit(X_train, y_train)
    return gs


C:\Users\Tomonobu\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [41]:
def train_rf_parameter(df):
    df = df[(df['Response'] == 0) | (df['Response'] == 1)]
    df_train, df_test = train_test_split(df, random_state=33)
    
    df_train_ok   = df_train[df_train['Response'] == 0]
    df_train_ng   = df_train[df_train['Response'] == 1]
    undersample_rate = len(df_train_ng) * 30 / len(df_train_ok)
    df_train_ok_sample = df_train_ok.sample(frac = undersample_rate)
    df_train = pd.concat([df_train_ok_sample, df_train_ng])

    rf = train_with_r_forest(df_train)
    X_test, y_test = separate_X_y(df_test)
    y_pred = rf.predict(X_test)
    print(y_pred)   
    
    cf_mat = confusion_matrix(y_test, y_pred)
    print(cf_mat)

    mcc = calc_mcc(cf_mat)
    print(mcc)    
    return rf

model = train_rf_parameter(df_all)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95 
31041 141 852 188
124717 11 572 3585
[CV]  colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95, total= 4.5min
[CV] colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.6min remaining:    0.0s


31060 122 859 181
124720 8 621 3536
[CV]  colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95, total= 4.6min
[CV] colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95 
31055 127 875 164
124718 10 676 3482
[CV]  colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95, total= 4.5min
[CV] colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95 
31061 121 859 180
124714 14 520 3638
[CV]  colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95, total= 4.2min
[CV] colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95 
31051 131 849 190
124718 10 660 3498
[CV]  colsample_bytree=1.0, max_depth=25, scale_pos_weight=2, subsample=0.95, total= 4.3min


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 22.4min finished


[0 0 0 ..., 0 0 0]
[[293022   1233]
 [  1371    311]]
293022 1233 1371 311
0.18856737844
